In [8]:
import pandas as pd
import dash
from dash import dcc, html
import plotly.express as px

# Lendo os arquivos Parquet
co2_by_region = pd.read_parquet('data-lakehouse/gold/co2_by_region.parquet')
top_countries_co2 = pd.read_parquet('data-lakehouse/gold/top_countries_co2.parquet')
co2_reduction_potential = pd.read_parquet('data-lakehouse/gold/reduction_potential_by_region.parquet')

# Verificando as primeiras linhas dos dataframes
print("Primeiras linhas de co2_by_region:", co2_by_region.head())
print("Primeiras linhas de top_countries_co2:", top_countries_co2.head())
print("Primeiras linhas de co2_reduction_potential:", co2_reduction_potential.head())

# Verificando as colunas dos dataframes
print("Colunas de co2_by_region:", co2_by_region.columns)
print("Colunas de top_countries_co2:", top_countries_co2.columns)
print("Colunas de co2_reduction_potential:", co2_reduction_potential.columns)

# Inicializando o app Dash
app = dash.Dash(__name__)

# Selecionando os 10 países com as maiores emissões de CO2
# Usando o nome correto da coluna: 'kilotons_of_co2_sum'
top_countries_co2 = top_countries_co2.nlargest(10, 'kilotons_of_co2_sum')

# Criando gráficos interativos com Plotly
fig1 = px.bar(co2_by_region, x='region', y='kilotons_of_co2_sum', title="Emissões de CO2 por Região")
fig2 = px.bar(top_countries_co2, x='country', y='kilotons_of_co2_sum', title="Top 10 Países por Emissões de CO2")
fig3 = px.bar(co2_reduction_potential, x='region', y='reduction_potential', title="Potencial de Redução de CO2 por Região")

# Definindo o layout do dashboard
app.layout = html.Div(children=[
    html.H1("Dashboard de Emissões de CO2", style={'textAlign': 'center'}),
    dcc.Graph(id='grafico1', figure=fig1),
    dcc.Graph(id='grafico2', figure=fig2),
    dcc.Graph(id='grafico3', figure=fig3),
])

# Rodando o servidor
if __name__ == '__main__':
    app.run(debug=True)


Primeiras linhas de co2_by_region:      region  kilotons_of_co2_sum  metric_tons_per_capita_sum  \
0    Europe         1.839814e+08                     8883.97   
1      Asia         3.705193e+08                     8600.92   
2  Americas         2.195244e+08                     3829.07   
3    Africa         2.884081e+07                     1668.13   
4   Oceania         1.151804e+07                     1573.80   

   reduction_potential  
0              8883.97  
1              8600.92  
2              3829.07  
3              1668.13  
4              1573.80  
Primeiras linhas de top_countries_co2:          country  kilotons_of_co2_sum  metric_tons_per_capita_sum
0          China         1.807228e+08                      136.52
1  United States         1.583179e+08                      544.55
2         Russia         5.002867e+07                      344.09
3          India         3.988273e+07                       33.62
4          Japan         3.507659e+07                      27